In [1]:
import pandas as pd
import re
import itertools
import cobra
import os
import collections
from cobra.io.dict import model_to_dict, model_from_dict, metabolite_from_dict, gene_from_dict, reaction_from_dict
from cobra import Metabolite, Reaction, Model, Gene
import multiprocessing
from multiprocessing import Process
import os, time, random
import sys
from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor
import threading

In [2]:
def get_solution():
    global feasible_solution
    feasible_solution = []

In [3]:
def reacid_metas():
    global dict_reacid_metas, reacid_metas_coeff, data1515_reac
    excel1515 = pd.ExcelFile('MCslimmodel1422.xlsx')
    data1515_reac = excel1515.parse('reactions', index_col=None).T.to_dict().values()
    dict_reacid_metas = {}
    reacid_metas_coeff = {}
    for line in data1515_reac:
        id = line['id']
        metabolites = line['metabolites']
        metabolites_split = metabolites.split("),")
        # print(metabolites_split)
        metalist = []
        metas_coeff = {}
        for meta_sp in metabolites_split:
            metabolites_values = re.search(r"'(.*)'", meta_sp).group()
            metalist.append(metabolites_values.replace("'", ""))
            metabolites_coefficient = re.search(r",(.*)", meta_sp).group()
            metabolites_coefficient = metabolites_coefficient.replace(",", "")
            metas_coeff[metabolites_values.replace("'", "")] = metabolites_coefficient
        reacid_metas_coeff[
            id] = metas_coeff

In [4]:
# 先正对碳个数间的转换进行计算
def meta_set_get(data1515_meta):
    meta_set = []
    for line in data1515_meta:
        formula = line['formula']
        cell = re.findall(r'([A-Z][a-z]*)(\d*)', str(formula))
        for one in cell:
            # print(one)
            c_num=0
            if 'C' in one:
                if one[1] == '':
                    c_num = int(1)
                else:
                    c_num = int(one[1])
            if c_num==1 or c_num==1 or c_num==2:#需要修改,按照目标的简化反应进行修改,如c1+c2=c3
                group = [c_num, line['id']]
                meta_set.append(group)
        # group = [c_num, line['id']]
        # meta_set.append(group)
    print(meta_set)
    return meta_set


# 功能函数：对列表进行去重
def uniq(a, b):
    for i in a:
        if i not in b:
            b.append(i)
    return b

# 建立全局空集flux_meta,reaction_tool
def summarize():
    global flux_meta, reaction_tool
    flux_meta = []
    reaction_tool = []

# 功能函数：判断两个词典列表是否相同
def compare(s, t):
    t = list(t)  # make a mutable copy
    try:
        for elem in s:
            t.remove(elem)
    except ValueError:
        return False
    return not t

In [5]:
def bubble_sort_advance(self):
    """
    冒泡排序改进算法，时间复杂度O(n^2)
    设置flag，当一轮比较中未发生交换动作，则说明后面的元素其实已经有序排列了。
    对于比较规整的元素集合，可提高一定的排序效率。
    """
    lis = self
    length = len(self)
    # print(length)
    i = 0
    while i < length:
        flag = False
        j = length - 2
        while j >= i:
            # print(lis[j], lis[j + 1])
            # print(j,j + 1)
            if lis[j][0] > lis[j + 1][0]:
                lis = swap(lis, j, j + 1)
                # print(lis)
            # elif lis[j][1] != lis[j + 1][0]:
            j -= 1
        i += 1
    return lis

In [6]:
def swap(lis, i, j):
    """定义一个交换元素的方法，方便后面调用。"""
    # print(lis)
    temp = lis[i]
    lis[i] = lis[j]
    lis[j] = temp
    # print(lis)
    return lis

In [7]:
def combination(l, n):
    l = list(sorted(filter(lambda x: x <= n, l)))
    # print("combination",l, n)
    combination_impl(l, n, [])

In [8]:
def combination_impl(l, n, stack):
    if n == 0 and len(stack) == 2:
        # print(stack,"stack",type(stack),len(stack))
        tool = []
        tool = stack.copy()
        # print(tool)
        feasible_solution.append(tool)
        # print("feasible_solution",feasible_solution)
        return
    for i in range(0, len(l)):
        if l[i] <= n:
            stack.append(l[i])
            combination_impl(l[i + 1:], n - l[i], stack)
            stack.pop()
        else:
            break

In [9]:
def iml1515meta_get():
    excel1515 = pd.ExcelFile('step3.xlsx')
    data1515_meta = excel1515.parse('metabolites', index_col=None).T.to_dict().values()
    return data1515_meta

In [10]:
def output_c1c1c2txt(need_fluxes, outputdir, a, b, model):
    need_fluxes_index=[]
    # 计算结果生成文本文件
    # need_fluxes = solution.fluxes[abs(solution.fluxes)>1e-10]  有通量的反应
    # outfile 生成文件的名称：‘D:/Work/1/result.txt'
    for need_id in need_fluxes.index:
        if "EX_" not in need_id:
            need_fluxes_index.append(need_id)
    print(len(need_fluxes_index))

    for need_id in need_fluxes.index:
        result_list = []
        rea = model.reactions.get_by_id(need_id)
        # need_fluxes[need_id]
        fl = str(round(need_fluxes[need_id], 5))
        outline_list = [need_id, fl, rea.reaction, str(rea.lower_bound), str(rea.upper_bound),
                        str(rea.objective_coefficient)]
        print("\t".join(outline_list))
    if len(need_fluxes_index)<=10:
        outfile =outputdir+ '656 {0} {1} {2} .txt'.format(b, a,len(need_fluxes_index))  # 02
        with open(outfile, 'w') as outf:
            for need_id in need_fluxes.index:
                result_list = []
                rea = model.reactions.get_by_id(need_id)
                print(need_id, need_fluxes[need_id], rea.reaction, rea.lower_bound, rea.upper_bound,rea.objective_coefficient)
                # need_fluxes[need_id]
                fl = str(round(need_fluxes[need_id], 5))
                outline_list = [need_id, fl, rea.reaction, str(rea.lower_bound), str(rea.upper_bound),str(rea.objective_coefficient)]
                outf.write("\t".join(outline_list) + '\n')
        print(outfile)

In [11]:
def summarize_reacandmeta():
    global id2reaceq, id2reacmetaid, simplifyreaction_list, simplifymeta_list, simplify_rm_list, cobra_meta
    id2reaceq = {}  # 简化模型中反应id和简化方程的对应关系
    excelid2r = pd.ExcelFile('step2.xlsx')
    dataid2r = excelid2r.parse('简化反应替换', index_col=None).T.to_dict().values()
    for line in dataid2r:
        id2reaceq[line['id']] = line['reaction_eq']
    cobra_meta = []  # FBA算得途径中的每个反应中的每个代谢物的集合list
    simplifyreaction_list = []  # FBA算得途径中的每个反应的重复反应集合（包括原反应）
    simplifymeta_list = []  # FBA算得途径中的每个代谢物的重复代谢物集合（包括原反应）
    simplify_rm_list = []  # FBA算得途径中的每个代谢物的相转换反应集合（包括原反应）
    # ==================================================================================
    for line in dataid2r:
        simplifyreaction_list.append(line)
    # ==================================================================================
    simplifymeta_list = excelid2r.parse('简化代谢物替换', index_col=None).T.to_dict().values()
    # ==================================================================================
    simplify_rm_list = excelid2r.parse('重复代谢物间的合并反应', index_col=None).T.to_dict().values()

    excelid2metaid = pd.ExcelFile('MCslimmodel1422.xlsx')
    dataid2metaid = excelid2metaid.parse('reactions', index_col=None).T.to_dict().values()
    id2reacmetaid = {}
    for line in dataid2metaid:
        id2reacmetaid[line['id']] = line['METACYC_id']

In [12]:
def add_meta(add_name, add_id, add_formula, model):#添加每个代谢物的交换反应
    for num in range(0, len(add_name)):
        reaction = Reaction('EX_{}'.format(add_id[num]))
        reaction.name = '{}'.format(add_name[num])
        reaction.formula = '{}'.format(add_formula[num])
        reaction.lower_bound = 0.  # This is the default
        reaction.upper_bound = 0.  # This is the default
        i = '{}'.format(add_id[num])
        s = model.metabolites.get_by_id("{}".format(i))
        reaction.add_metabolites({s: -1.0, })
        model.add_reactions([reaction])
    a = model_to_dict(model, sort=False)
    return model

def add_2product_exchangge(product1,product2,model):
    reaction = Reaction('EX_{0}+{1}'.format(product1,product2))
    reaction.name = '{0}+{1}'.format(product1,product2)
    reaction.lower_bound = -1000.  # This is the default
    reaction.upper_bound = 1000.  # This is the default
    if product1==product2:
        s = model.metabolites.get_by_id("{}".format(product1))
        reaction.add_metabolites({s: -1.0, })
    else:
        s1 = model.metabolites.get_by_id("{}".format(product1))
        s2 = model.metabolites.get_by_id("{}".format(product2))
        reaction.add_metabolites({s1: -1.0, s2: -1.0 })
    model.add_reactions([reaction])
    return model

def GetNullFileAndDir(filepath):
    delpath = []
    for root, dirs, files in os.walk(filepath):
        for name in files:
            path = os.path.join(root, name)
            try:
                if not os.listdir(path):
                    print(path)
                    delpath.append(path)
            except:
                size = os.path.getsize(path)
                if size == 0:
                    print(path)
                    delpath.append(path)

        for name in dirs:
            path = os.path.join(root, name)
            try:
                if not os.listdir(path):
                    print(path)
                    delpath.append(path)
            except:
                size = os.path.getsize(path)
                if size == 0:
                    print(path)
                    delpath.append(path)
        return delpath


def compare2(s, t):
    return sorted(s) == sorted(t)


def DelFileAndDir(delpath):
    for path in delpath:
        try:
            os.rmdir(path)
        except:
            os.remove(path)


def model_reacbounds_set(model2, twain, product, c1_coefficient_a, c1_coefficient_b):  # ,c2_coefficient
    model2.reactions.get_by_id('EX_N1_1').bounds = (-1000, 1000)
    # print(twain[0],twain[1])
    c1meta_a = twain[0][1]
    c1meta_b = twain[1][1]
    c2meta = product
    # print(c1meta_a,twain[0][0],"/"*20,c1meta_b,twain[1][0],"/"*20,c2meta)
    # print("c1_coefficient_a",c1_coefficient_a)
    # print("c1_coefficient_b",c1_coefficient_b)
    # sys.exit(0)
    if twain[0] == twain[1]:  # c1meta_a和c1meta_b的相同可视作为c1_coefficient_a和c1_coefficient_b的相同
        model2.reactions.get_by_id('EX_{}'.format(twain[0][0])).bounds = (-120, -120)
        # print("代谢物相同 反应相同")
        if float(c1_coefficient_a) < 0:
            model2.reactions.get_by_id('{}'.format(c1meta_a)).bounds = (120, 120)
        else:
            model2.reactions.get_by_id('{}'.format(c1meta_a)).bounds = (-120, -120)
    else:
        # print("代谢物相同 反应不同")

        if twain[0][0] == twain[1][0]:
            model2.reactions.get_by_id('EX_{}'.format(twain[0][0])).bounds = (-120, -120)
        else:
            model2.reactions.get_by_id('EX_{}'.format(twain[0][0])).bounds = (-60, -60)
            model2.reactions.get_by_id('EX_{}'.format(twain[1][0])).bounds = (-60, -60)

        if float(c1_coefficient_a) < 0:
            model2.reactions.get_by_id('{}'.format(c1meta_a)).bounds = (60, 60)  # 设定输入反应的下限
        else:
            model2.reactions.get_by_id('{}'.format(c1meta_a)).bounds = (-60, -60)

        if float(c1_coefficient_b) < 0:
            model2.reactions.get_by_id('{}'.format(c1meta_b)).bounds = (60, 60)  # 设定输入反应的下限
        else:
            model2.reactions.get_by_id('{}'.format(c1meta_b)).bounds = (-60, -60)

    # if c2_coefficient < 0:
    #     model2.reactions.get_by_id('{}'.format(product[1])).bounds = (-60, -60)  # 设定输入反应的下限
    # else:
    #     model2.reactions.get_by_id('{}'.format(product[1])).bounds = (60, 60)
    # model2.reactions.get_by_id('{}'.format(product[1])).bounds = (0, 1000)

    # model2.reactions.get_by_id('R_254').bounds = (0, 0)
    # model2.reactions.get_by_id('R_4736').bounds = (0, 1000)
    model2.reactions.get_by_id('EX_{}'.format(product)).bounds = (0, 1000)
    model2.objective = "EX_{}".format(product)  # 设定目标函数
    return model2


def print_txtfile(score, twain, product, fba_solution, need_fluxes_index, model2, outputdir):
    # print(
    #     '{0}-{1}-{2} reac:{3}-{4} fba: {5}'.format(twain[0][0], twain[1][0], product, twain[0][1], twain[1][1],
    #                                                fba_solution.fluxes["EX_{}".format(product)]))

    need_fluxes = fba_solution.fluxes[abs(fba_solution.fluxes) > 1e-10]  # abs() 函数返回数字的绝对值。
    for need_id in need_fluxes.index:
        if "EX_" not in need_id:
            need_fluxes_index.append(need_id)
    print("need_fluxes_index", len(need_fluxes_index), need_fluxes_index)
    print(need_fluxes.index)

    outfile = outputdir + str(
        '{0} {1}-{2}-{3} reac{4}-{5} fba={6} number={7}.txt'.format(
            score, twain[0][0], twain[1][0], product, twain[0][1], twain[1][1],
            '%.2f' % fba_solution.fluxes["EX_{}".format(product)], len(need_fluxes_index)
        )
    )

    # print(len(need_fluxes), need_fluxes)
    # print(outfile)
    # output_txt(need_fluxes, outfile, [twain[0], twain[1]], product, model2)  # 根据FBA算得的解，对所得循环途径进行整理和输出txt文件


def print_excelfile(score, twain, product, fba_solution, need_fluxes_index, outputdir):
    aftercobra_meta = []
    cobra_meta2 = cobra_meta + [twain[0], twain[1], product]
    aftercobra_meta = uniq(cobra_meta2, aftercobra_meta)
    # print('aftercobra_meta', aftercobra_meta[:3])
    afte_simplifymeta, afte_simplify_rm = extra_metabolites_manage(aftercobra_meta)
    # print('afte_simplifymeta', afte_simplifymeta[:3])
    # print('afte_simplify_rm', afte_simplify_rm[:3])
    # print(flux_meta[:3])
    # print(reaction_tool[:3])
    print(fba_solution.fluxes["EX_{}".format(product)])
    if fba_solution.fluxes["EX_{}".format(product)] > 50:
        print_reaction_tool = []
        print_reaction_tool = uniq(reaction_tool, print_reaction_tool)
        writer = pd.ExcelWriter(
            outputdir + (
                '{0} {1}-{2}-{3} reac{4}-{5} fba={6} number={7}.xlsx'.format(score, twain[0][0], twain[1][0],
                                                                             product, twain[0][1],
                                                                             twain[1][1],
                                                                             '%.2f' % fba_solution.fluxes[
                                                                                 "EX_{}".format(product)],
                                                                             len(need_fluxes_index))))

        pd.DataFrame(flux_meta).to_excel(writer, '可行FBA计算通量总结', index=True)
        pd.DataFrame(print_reaction_tool).to_excel(writer, '算得途径中所有反应和重复反应', index=True)  # 还有问题
        pd.DataFrame(afte_simplifymeta).to_excel(writer, '算得途径中的每个代谢物的重复代谢物', index=True)
        pd.DataFrame(afte_simplify_rm).to_excel(writer, '同简化形式的重复代谢物间的转换反应', index=True)
        writer.save()

In [13]:
def calculate_main(inputc1c2c3, model, outputdir):
    # twain=['C1_13','C1N1_3']
    # product='C2_10'
    # sys.exit(0)
    need_fluxes_index = []
    summarize()
    summarize_reacandmeta()
    twain = inputc1c2c3[0]
    print(inputc1c2c3)
    # print(twain)
    product = inputc1c2c3[1]
    for line in twain:
        print(line[0])
        model.reactions.get_by_id('EX_{}'.format(line[0])).bounds = (-1000, 1000)
    model.reactions.get_by_id('EX_N1_1'.format(product)).bounds = (-1000, 1000)
    model.reactions.get_by_id("EX_{}".format(product)).bounds = (-1000, 1000)
    #先设定反应物的输入和氮的输入
    #添加 c1+c1--> 这样两个产物组成的交换反应 ,为了保证可变性不添加到原模型中
    with model as model2:
        if twain[0] == twain[1]:
            print("same")
            model2.reactions.get_by_id('EX_{}'.format(twain[0][0])).bounds = (-120,-120)  # -120
            # model.reactions.get_by_id("cbR_254").bounds = (60, 60)
            # model.reactions.get_by_id("R_552").bounds = (60, 60)
        else:
            model2.reactions.get_by_id('EX_{}'.format(twain[0][0])).bounds = (-60, -60)
            model2.reactions.get_by_id('EX_{}'.format(twain[1][0])).bounds = (-60, -60)
        #添加反应限制
#         model2.reactions.get_by_id('CS').bounds = (0,0)
#         model2.reactions.get_by_id('FBA').bounds = (0, 0)
        # model2.reactions.get_by_id('GART').bounds = (0, 0)
        # model2.reactions.get_by_id('GHMT2r').bounds = (0, 0)
        '''
        try:
            model2.objective = "EX_{}".format(product)   # 设定目标函数
            pfba_solution = cobra.flux_analysis.pfba(model2)
            need_fluxes = pfba_solution.fluxes[abs(pfba_solution.fluxes) > 1e-10]  # abs() 函数返回数字的绝对值。
            print(need_fluxes,len(need_fluxes))
            print(need_fluxes.index,len(need_fluxes))
            output_c1c1c2txt(need_fluxes, outputdir, [twain[0][0], twain[1][0]], product, model2)
        except:
            print('{0}+{1}->{2}---error'.format(twain[0][0],twain[1][0],product))
        '''
        model2.objective = "EX_{}".format(product)   # 设定目标函数
        pfba_solution = cobra.flux_analysis.pfba(model2)
        need_fluxes = pfba_solution.fluxes[abs(pfba_solution.fluxes) > 1e-10]  # abs() 函数返回数字的绝对值。
        print(need_fluxes,len(need_fluxes))
        print(need_fluxes.index,len(need_fluxes))
        output_c1c1c2txt(need_fluxes, outputdir, [twain[0][0], twain[1][0]], product, model2)

In [14]:
if __name__ == '__main__':
    reacid_metas()
    model = cobra.io.read_sbml_model("MCslimmodel1422_new.xml")  # ('D:/Work/1/textbook.xml')输入模型
    # 输入：生成的简化模型的路径和文件名
    outputdir = "/media/jupyter/zhang_rzh/slimGEM(MC)/calculate/result/"  # c1-c1-c2 003c1c1c2_反应组合，需要修改
    # 输出：算得途径的txt文件的输出文件夹路径
    add_list = ['N1_1']  # 添加氮源
    # del_file(outputdir)  # 清空输出的文件夹
    data1515_meta = iml1515meta_get()
    aaa = meta_set_get(data1515_meta)  # 需要修改的地方
    lis = bubble_sort_advance(aaa)
    print(lis, "lis", len(lis))
    num = 0

    NUM = []
    NUM2 = []
    NUM2_sum = []
    for line in lis:  # (可更换lis，lis==1515简化模型的碳一代谢物i列表)
        NUM.append(line[0])
    uniq(NUM, NUM2)
    NUM2_sum = NUM2 + NUM2

    # 开始求解n的合并组合
    o = 0
    a = model_to_dict(model, sort=False)
    meta = a['metabolites']
    add_id = []
    add_name = []
    add_formula = []
    print(NUM2)
    for i in range(1, len(NUM2)):  # 筛选出二碳代谢物列表
        i = 1  # 需要带动的地方，目标代谢物的
        print(NUM2[i])
        # 收集目标代谢物
        targetprod_set = []
        c2list = []
        for line in lis:
            if line[0] == NUM2[i]:
                targetprod_set.append(line[1])
        c2list = targetprod_set.copy()

        print(len(targetprod_set), "c2list", targetprod_set[:5])
        get_solution()
        combination(NUM2_sum, NUM2[i])
        feasi_sol = []
        uniq(feasible_solution, feasi_sol)
        target1 = []
        target2 = []
        c1list_a = []
        c1list_b = []
        reaidlist = []
        for pair in feasi_sol:
            print(pair)
            for line in lis:
                if line[0] == pair[0]:
                    tool1 = [line[1]]
                    target1.append(tool1)
                    c1list_a.append(line[1])
            for line in lis:
                if line[0] == pair[1]:
                    tool1 = [line[1]]
                    target2.append(tool1)
                    c1list_b.append(line[1])
        print(len(target1), "c1list_a", target1[:5])
        print(len(target2), "c1list_b", target2[:5])
        print(len(lis), lis[:5])
        reaidlist02 = []
        targetpair = []
        targetpair = list(itertools.product(target1, target2))
        tool = targetpair.copy()
        targetpair2 = []
        for line in targetpair:
            v = 0
            tool.remove(line)
            for i in tool:
                if compare2(line, i):
                    v = 1
            if v == 0:
                targetpair2.append(line)
        print(len(targetpair2), "碳一组合", targetpair2[:5])
        all_list = c1list_a + c1list_b + c2list + add_list
        resultlist = []
        resultlist = uniq(all_list, resultlist)
        input_a = []
        input_b = []
        output = []
        input_a = uniq(target1, input_a)
        input_b = uniq(target2, input_b)
        output = uniq(targetprod_set, output)
        for line in meta:
            if line['id'] in resultlist:
                add_id.append(line['id'])
                add_name.append(line['name'])
                add_formula.append(line['formula'])
        model = add_meta(add_name, add_id, add_formula, model)#添加交换反应
        output = c2list
        c3feasipair = list(itertools.product(targetpair2, output))
        print(len(c3feasipair), c3feasipair[:5])
        pList = []
        for i in c3feasipair:
            print("i=",i)
#             i= ((['C1_1'], ['C1_1']), 'C2_2')
            calculate_main(i, model, outputdir)
            break
        break

cobra/io/sbml3.py:205 UserWarning: https://identifiers.org/bigg.metabolite/XYLULOSE__45__5__45__PHOSPHATE_c does not start with http://identifiers.org/
cobra/io/sbml3.py:205 UserWarning: https://identifiers.org/bigg.metabolite/XTP_c does not start with http://identifiers.org/
cobra/io/sbml3.py:205 UserWarning: https://identifiers.org/bigg.metabolite/XANTHINE_p does not start with http://identifiers.org/
cobra/io/sbml3.py:205 UserWarning: https://identifiers.org/bigg.metabolite/SULFOACETALDEHYDE_c does not start with http://identifiers.org/
cobra/io/sbml3.py:205 UserWarning: https://identifiers.org/bigg.metabolite/VANILLIN_c does not start with http://identifiers.org/
cobra/io/sbml3.py:205 UserWarning: https://identifiers.org/bigg.metabolite/VALERATE_c does not start with http://identifiers.org/
cobra/io/sbml3.py:205 UserWarning: https://identifiers.org/bigg.metabolite/VAL_p does not start with http://identifiers.org/
cobra/io/sbml3.py:205 UserWarning: https://identifiers.org/bigg.metab

[[2, 'C2_1'], [1, 'C1N2_1'], [2, 'C2_10'], [2, 'C2_11'], [2, 'C2_12'], [2, 'C2_13'], [2, 'C2_14'], [2, 'C2_15'], [2, 'C2_16'], [2, 'C2_17'], [2, 'C2_18'], [2, 'C2_19'], [2, 'C2_2'], [2, 'C2_20'], [2, 'C2_21'], [2, 'C2_22'], [2, 'C2_23'], [2, 'C2_3'], [2, 'C2_4'], [2, 'C2_5'], [2, 'C2_6'], [2, 'C2_7'], [2, 'C2_8'], [2, 'C2_9'], [1, 'C1_1'], [1, 'C1N1_1'], [1, 'C1N1_2'], [1, 'C1_2'], [1, 'C1_3'], [1, 'C1_4'], [1, 'C1_5'], [1, 'C1_6'], [1, 'C1_7'], [1, 'C1_8'], [1, 'C1N1_3'], [1, 'C1_9'], [1, 'C1N1_4'], [1, 'C1N3_1'], [1, 'C1_10'], [1, 'C1_11'], [1, 'C1N2_2'], [1, 'C1_12'], [1, 'C1_13'], [2, 'C2N1_1'], [2, 'C2N1_10'], [2, 'C2N1_11'], [2, 'C2N1_12'], [2, 'C2N1_13'], [2, 'C2N1_14'], [2, 'C2N1_15'], [2, 'C2N1_16'], [2, 'C2N1_17'], [2, 'C2N1_2'], [2, 'C2N1_3'], [2, 'C2N1_4'], [2, 'C2N1_5'], [2, 'C2N1_6'], [2, 'C2N1_7'], [2, 'C2N1_8'], [2, 'C2N1_9'], [2, 'C2N2_1'], [2, 'C2N2_2'], [2, 'C2N2_3'], [2, 'C2N2_4'], [2, 'C2N3_1'], [2, 'C2N3_2'], [2, 'C2N3_3'], [2, 'C2N3_4'], [2, 'C2N3_5'], [2, 'C2N4_